<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
import time
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import shutil
from google.colab import files

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%%time
!rm -r /content/cloned-repo
%cd ..
!git clone -l -s git://github.com/kavyajeetbora/EAST.git /content/cloned-repo
%cd /content/cloned-repo
!ls

/
Cloning into '/content/cloned-repo'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 169 (delta 48), reused 0 (delta 0), pack-reused 82
Receiving objects: 100% (169/169), 6.91 MiB | 10.18 MiB/s, done.
Resolving deltas: 100% (93/93), done.
/content/cloned-repo
dataset.py		    EAST_training_2.ipynb	    model.py
detect.py		    EAST_training.ipynb		    model_resnet.py
dsiplay_ground_truth.ipynb  EAST_training_vgg16_bn_2.ipynb  train.py
EAST_detection.ipynb	    EAST_training_vgg16_bn.ipynb
EAST_resnet.py		    loss.py
CPU times: user 85.4 ms, sys: 31.4 ms, total: 117 ms
Wall time: 7.19 s


In [0]:
from dataset import *

In [7]:
%%time

# AI4Bharat dataset
image_folder = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Images/Images.Zip (Unzipped Files)'
label_folder = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Annotations/Annotations.Zip (Unzipped Files)'

training_dataset = custom_dataset(image_folder, label_folder)
print(len(training_dataset))

5000
CPU times: user 20 ms, sys: 2.3 ms, total: 22.3 ms
Wall time: 76.3 ms


## Loss for one batch

In [8]:
%%time
train_loader = DataLoader(training_dataset, shuffle = True, batch_size=3, drop_last=True)
imgs, gt_score, gt_geo, ignored_maps = next(iter(train_loader))
print(imgs.size(), gt_score.size(), gt_geo.size(), ignored_maps.size())
print(torch.sum(gt_score))

torch.Size([3, 3, 512, 512]) torch.Size([3, 1, 128, 128]) torch.Size([3, 5, 128, 128]) torch.Size([3, 1, 128, 128])
tensor(1134.)
CPU times: user 1.18 s, sys: 1.55 s, total: 2.73 s
Wall time: 4 s


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [10]:
%%time
from model import EAST
model = EAST()
pretrained_path = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/EAST scene text detector/east_vgg16.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
model = model.to(device)
imgs, gt_score, gt_geo, ignored_maps = imgs.to(device), gt_score.to(device), gt_geo.to(device), ignored_maps.to(device)
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

torch.Size([3, 1, 128, 128]) torch.Size([3, 5, 128, 128])
CPU times: user 5.91 s, sys: 1.25 s, total: 7.16 s
Wall time: 7.12 s


In [11]:
from loss import Loss

loss_fn = Loss()
loss_fn(gt_score, pred_score, gt_geo, pred_geo, ignored_maps)

classify loss is 0.63155860, angle loss is 0.01209032, iou loss is 0.55790365


tensor(1.3104, device='cuda:0', grad_fn=<AddBackward0>)

## Training Full Batch 

fine tuning pretrained model [east_vgg16.pth](https://drive.google.com/file/d/1AFABkJgr5VtxWnmBU3XcfLJvpZkC2TAg/view)

In [0]:
model = EAST()
pretrained_path = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/EAST scene text detector/east_7_11_19.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
model = model.to(device)

In [0]:
def train(model, train_img_path, train_gt_path, batch_size, lr, epoch_iter, num_workers=4, interval=1):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=epoch_iter//2, gamma=0.5)
    loss_curve = []
    
    for epoch in range(epoch_iter):
      model.train()
      scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      
      
      if (epoch + 1) % interval == 0:
        state_dict = model.state_dict()
        torch.save(state_dict, 'model_epoch_{}.pth'.format(epoch+1))
        try:
          files.download('model_epoch_{}.pth'.format(epoch+1))
          print('Load Successfully downloaded')
        except:
          print('Couldn\'t download the weights')
          pass

      ## plotting the loss curve
      loss_curve.append(epoch_loss/int(file_num/batch_size))

    return loss_curve

In [0]:
%%time
loss_curve = train(model, image_folder, label_folder, batch_size=3, lr=1e-5, epoch_iter=5, interval = 1)

5000


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


classify loss is 0.30393267, angle loss is 0.00122857, iou loss is 0.36814147
Epoch is [1/5], mini-batch is [1/1666], time consumption is 0.12240624, batch_loss is 0.68435979
classify loss is 0.43647248, angle loss is 0.00833047, iou loss is 0.58183515
Epoch is [1/5], mini-batch is [2/1666], time consumption is 0.10155535, batch_loss is 1.10161233
classify loss is 0.17588258, angle loss is 0.00196113, iou loss is 0.29094490
Epoch is [1/5], mini-batch is [3/1666], time consumption is 0.10711312, batch_loss is 0.48643881
classify loss is 0.28157961, angle loss is 0.00387654, iou loss is 0.34792185
Epoch is [1/5], mini-batch is [4/1666], time consumption is 0.10035729, batch_loss is 0.66826689
classify loss is 0.32340068, angle loss is 0.00297893, iou loss is 0.28064114
Epoch is [1/5], mini-batch is [5/1666], time consumption is 0.15420651, batch_loss is 0.63383114
classify loss is 0.27650815, angle loss is 0.00592364, iou loss is 0.34254763
Epoch is [1/5], mini-batch is [6/1666], time co

In [0]:
plt.plot(loss_curve)

1. [**Adam and AdamW algorithms**](https://www.fast.ai/2018/07/02/adam-weight-decay/)

2. [**The optimal learning rate during fine-tuning of an artificial neural network**](https://www.mikulskibartosz.name/the-optimal-learning-rate-during-fine-tuning-of-an-artificial-neural-network/)